In [ ]:
from hardware_testing.gravimetric.helpers import get_api_context
from opentrons_shared_data.liquid_classes.liquid_class_definition import PositionReference
from opentrons.protocol_api import InstrumentContext

In [ ]:
# DEFINE SLOTS

SLOTS = {
    "trash": "A3",
    "t50": "B1",
    "t200": "B2",
    "t1000": "B3",
    "water": "D1",
    "glycerol_50": "D2",
    "ethanol_80": "D3",
}

In [ ]:
# PROTOCOL CONTEXT (skip LPC offsets)

ctx = get_api_context(include_labware_offsets=False)

In [ ]:
# SETUP LIQUIDS + VIALS

ASSUMED_STARTING_VIAL_VOLUME = 1200
LIQUID_NAMES = ["water", "glycerol_50", "ethanol_80"]

LIQUIDS = {
    n: {
        "class": ctx.define_liquid_class(n),
        "label": ctx.define_liquid(n),
    }
    for n in LIQUID_NAMES
}
VIALS = {
    n: ctx.load_labware(
        load_name="radwag_pipette_calibration_vial",
        location=SLOTS[n],
        version=2
    )["A1"]  # NOTE: storing the Well (not the Labware)
    for n in LIQUID_NAMES
}
for n in LIQUID_NAMES:
    VIALS[n].load_liquid(LIQUIDS[n]["label"], ASSUMED_STARTING_VIAL_VOLUME)

In [ ]:
# SETUP TIPS

STARTING_TIP = "A1"

TIP_RACKS = {
    tip: ctx.load_labware(
        load_name=f"opentrons_flex_96_tiprack_{tip}ul",
        location=SLOTS[f"t{tip}"]
    )
    for tip in [50, 200, 1000]
}
ctx.load_trash_bin(SLOTS["trash"])

In [ ]:
# SETUP PIPETTES

PIPETTES = {
    50: ctx.load_instrument(
        instrument_name="flex_1channel_50",
        mount="left",
    ),
    1000: ctx.load_instrument(
        instrument_name="flex_1channel_1000",
        mount="right",
    ),
}

In [ ]:
# SETUP TRIAL SEQUENCE

VOLUMES = {
    50: {
        50: [1, 10, 50],
    },
    1000: {
        50: [5, 10, 50],
        200: [5, 50, 200],
        1000: [10, 100, 1000],
    }
}


def _transfer_liquid_qc_volumes(liquid_name: str, pipette: int, tip: int) -> None:
    _pip = PIPETTES[pipette]
    _vial = VIALS[liquid_name]
    if _vial.current_liquid_volume() == ASSUMED_STARTING_VIAL_VOLUME:
        _pip.liquid_presence_detection = _vol_is_at_default
    volumes = VOLUMES[pipette][tip]
    _pip.tip_racks = [TIP_RACKS[tip]]
    for v in volumes:
        _pip.transfer_liquid(
            liquid_class=LIQUIDS[liquid_name]["class"],
            volume=v,
            source=_vial,
            dest=_vial,
            new_tip="once"
        )
        _pip.liquid_presence_detection = False  # always disable LLD


In [ ]:
raise RuntimeError("don't get careless and hit Run All")

In [ ]:
# HOME (optional)

ctx.home()

In [ ]:
# RETRACT PIPETTES

PIPETTES[50]._retract()
PIPETTES[1000]._retract()

In [ ]:
# PICKUP TIP + PROBE LIQUID

_probing_pip = PIPETTES[50]  # NOTE: P50 is most accurate
for n in LIQUID_NAMES:
    _probing_pip.pick_up_tip()
    _probing_pip.detect_liquid_presence(VIALS[n])
    _probing_pip.drop_tip()

In [ ]:
# P50 + T50

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=50, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=50, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=50, tip=50)

In [ ]:
# P1000 + T50

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=50)

In [ ]:
# P1000 + T200

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=200)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=200)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=200)

In [ ]:
# P1000 + T1000

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=1000)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=1000)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=1000)